In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 201c9f46-3f1c-466c-a636-e411020653c9, 23, Finished, Available, Finished)

In [11]:
df = spark\
    .read\
    .format("csv")\
    .option("header", "true")\
    .load("Files/adventureworks/AdventureWorks_Calendar.csv")

display(df)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 91d7c0c0-69bd-433b-b5ca-093bac6d1bbd)

In [13]:
df = df.withColumn("Date", to_date("Date", "M/d/y"))
display(df)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b3c01c64-36a3-4980-a7f6-f6371e661443)

In [21]:
%%sql

DROP TABLE IF EXISTS dbo.dimcalendar

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 23, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [22]:
df = df.withColumn("Year", year(df["Date"]))\
    .withColumn("MonthNumber", month(df["Date"]))\
    .withColumn("MonthName", date_format(df["Date"], "MMM"))\
    .withColumn("Quarter", date_format(df["Date"], "QQQ"))\
    .withColumn("DayOfWeek", date_format(df["Date"], "E"))\
    .withColumn("DayOfMonth", date_format(df["Date"], "d"))

df = df.withColumn("DayOfMonth", col("DayOfMonth").cast(IntegerType()))

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2579e597-7e9f-43b3-8a67-e70fb58727af)

In [23]:
df.write.mode("overwrite").format("delta").saveAsTable("DimCalendar")

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 25, Finished, Available, Finished)

# Load of Dim Products

In [70]:
df_categories = spark.read.format("csv").option("header","true").load("Files/adventureworks/AdventureWorks_Product_Categories.csv")
# df now is a Spark DataFrame containing CSV data from "Files/adventureworks/AdventureWorks_Product_Categories.csv".
display(df_categories)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 72, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e8201f76-a6f3-46f8-b47d-dff64f95c378)

In [71]:
df_categories = df_categories\
    .withColumn("ProductCategoryKey", col("ProductCategoryKey").cast(IntegerType()))
display(df_categories)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 73, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1b274386-0520-457d-8516-a43abbe7b61c)

In [72]:
df_subcategories = spark.read.format("csv").option("header","true").load("Files/adventureworks/AdventureWorks_Product_Subcategories.csv")
# df now is a Spark DataFrame containing CSV data from "Files/adventureworks/AdventureWorks_Product_Subcategories.csv".
display(df_subcategories)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 74, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4044eecb-b5c5-401f-9f36-86285b7d69ce)

In [73]:
df_subcategories = df_subcategories\
    .withColumn("ProductSubcategoryKey", col("ProductSubcategoryKey").cast(IntegerType()))\
    .withColumn("ProductCategoryKey", col("ProductCategoryKey").cast(IntegerType()))

display(df_subcategories)


StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 75, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d2cd4f86-965c-4e24-9008-19a0b30caf1e)

In [74]:
df_products = spark.read.format("csv").option("header","true").load("Files/adventureworks/AdventureWorks_Products.csv")
# df now is a Spark DataFrame containing CSV data from "Files/adventureworks/AdventureWorks_Products.csv".
display(df_products)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 76, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, adb11193-8638-4f77-8752-5c96ef8945aa)

In [75]:
df_products = df_products\
    .withColumn("ProductKey", col("ProductKey").cast(IntegerType()))\
    .withColumn("ProductSubcategoryKey", col("ProductSubcategoryKey").cast(IntegerType()))\
    .withColumn("ProductCost", col("ProductCost").cast(DecimalType(12, 2)))\
    .withColumn("ProductPrice", col("ProductPrice").cast(DecimalType(12, 2)))

display(df_products)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 77, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 58b86984-1b62-4109-a1ec-ee089389a802)

## J

### Join Categories, Subcategories and products tables

In [85]:
df_cat_sub_prod = df_categories\
    .join(df_subcategories, df_categories.ProductCategoryKey == df_subcategories.ProductCategoryKey, "inner")\
    .join(df_products, df_subcategories.ProductSubcategoryKey == df_products.ProductSubcategoryKey, "inner")

display(df_cat_sub_prod)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 87, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8e02da11-9c39-4a1e-8cbe-8934ba22bdfb)

In [86]:
df_cat_sub_prod = df_cat_sub_prod\
    .select(
        col("ProductKey"),
        col("CategoryName"),
        col("SubcategoryName"),
        col("ProductName"),   
        col("ProductSKU"),
        col("ModelName"),
        col("ProductDescription"),
        col("ProductColor"),
        col("ProductSize"),
        col("ProductStyle"),
        col("ProductCost"),
        col("ProductPrice")
    )

display(df_cat_sub_prod)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 88, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2888019e-8ab5-4d24-9572-489ec2d5afe0)

In [87]:
spark.sql("DROP TABLE IF EXISTS dimproducts")

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 89, Finished, Available, Finished)

DataFrame[]

In [88]:
df_cat_sub_prod.write.mode("overwrite").format("delta").saveAsTable("DimProducts")

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 90, Finished, Available, Finished)

In [89]:
df = spark.sql("SELECT * FROM Lakehouse001.dbo.dimproducts LIMIT 1000")
display(df)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 91, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 44265a61-b9b9-4fec-b0c9-ab56564cc585)

# Load of Fact Sales

# Load 

In [90]:
df_sales = spark.read.format("csv").option("header","true").load("Files/adventureworks/AdventureWorks_Sales_*.csv")

display(df_sales)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 92, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6bd387da-6c36-4e17-9d27-d2ef43f4bbc1)

In [92]:
df_sales = df_sales.withColumn("OrderDate", to_date("OrderDate", "M/d/y"))\
    .withColumn("StockDate", to_date("StockDate", "M/d/y"))\
    .withColumn("ProductKey", col("ProductKey").cast(IntegerType()))\
    .withColumn("CustomerKey", col("CustomerKey").cast(IntegerType()))\
    .withColumn("TerritoryKey", col("TerritoryKey").cast(IntegerType()))\
    .withColumn("OrderLineItem", col("OrderLineItem").cast(IntegerType()))\
    .withColumn("OrderQuantity", col("OrderQuantity").cast(IntegerType()))\
    .withColumn("UnitPrice", col("UnitPrice").cast(DecimalType(12, 2)))

display(df_sales)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 94, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 65eb6dfe-1439-4cc9-b51a-20534e056282)

In [93]:
df_sales.write.mode("overwrite").format("delta").saveAsTable("Sales")

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 95, Finished, Available, Finished)

In [94]:
df = spark.sql("SELECT * FROM Lakehouse001.dbo.sales LIMIT 1000")
display(df)

StatementMeta(, 99073465-5d41-4244-9964-3e963978a3da, 96, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b2046a42-057a-4817-b3bd-afbf7d1a0dac)

# Load of Dim Customers

In [34]:
df_customers = spark.read.format("csv").option("header","true").load("Files/adventureworks/AdventureWorks_Customers.csv")
# df now is a Spark DataFrame containing CSV data from "Files/adventureworks/AdventureWorks_Customers.csv".
display(df_customers)

StatementMeta(, 201c9f46-3f1c-466c-a636-e411020653c9, 36, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 006e8aeb-5a6c-4cfa-abf4-75b04ac4f3c3)

In [35]:
df_customers = df_customers\
    .withColumn("CustomerKey", col("CustomerKey").cast(IntegerType()))\
    .withColumn("BirthDate", to_date("BirthDate", "M/d/y"))\
    .withColumn("AnnualIncome", regexp_replace("AnnualIncome", "\$|,", "").cast(DecimalType(12,2)))\
    .withColumn("TotalChildren", col("TotalChildren").cast(IntegerType()))

display(df_customers)

StatementMeta(, 201c9f46-3f1c-466c-a636-e411020653c9, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5d3072b9-d8d5-44ed-986a-965b0b1b5a6c)

In [ ]:
df_customers.write.mode("overwrite").format("delta").saveAsTable("dimcustomers")